grasp

In [1]:
import genesis as gs
import numpy as np
import torch
from pytransform3d import (
    transformations as pt,
    rotations as pr,
    batch_rotations as pb,
    trajectories as ptr,
    plot_utils as ppu
)
from pandaSim.geometry.genesis_adapter import GenesisAdapter
from pandaSim.planning.screw_motion_planner import ScrewMotionPlanner


import spatialmath as sm
# auto reaload
%load_ext autoreload
%autoreload 2

[I 05/13/25 21:02:51.677 26143] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:

gs.init(backend=gs.gpu)
gs.set_random_seed(seed=42)
adapter = GenesisAdapter()
planner = ScrewMotionPlanner()

[Genesis] [21:02:54] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [21:02:54] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [21:02:54] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [21:02:55] [INFO] Running on [NVIDIA GeForce RTX 3070 Laptop GPU] with backend gs.cuda. Device memory: 7.78 GB.
[Genesis] [21:02:55] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [21:02:55] [INFO] Scene <baa42ad> created.


In [3]:
scene = adapter.scene
plane = scene.add_entity(
    gs.morphs.Plane(),
)
# robot_path = "../assets/urdf/franka_description/robots/frankaEmikaPandaVirtual.urdf"
# franka_virtual = scene.add_entity(
#     gs.morphs.URDF(file=robot_path, fixed=True),
# )

franka_MJCF = scene.add_entity(
    gs.morphs.MJCF(file="xml/franka_emika_panda/panda.xml",
                   pos = (0.0, 2.0, 0.0)),
    
)
franka = scene.add_entity(
    gs.morphs.MJCF(file="../assets/xml/franka_emika_panda/panda.xml")
                   
)
cube_size = (0.1, 0.07, 0.03)
cube_pos = [(0.7, 0, 0.015), (0.7, 2.0, 0.015), (0.7, -2.0, 0.015)]
cube, cube2, cube3 = [scene.add_entity(
    gs.morphs.Box(
        size = cube_size,
        pos  = pos,
    ),
    surface=gs.surfaces.Default(
        color=(0.8, 0.1, 0.1),

    )
    
) for pos in cube_pos]

target_entity = scene.add_entity(
    gs.morphs.Mesh(
        file="meshes/axis.obj",
        scale=0.15,
        collision=False,
    ),
    surface=gs.surfaces.Default(color=(1, 0.5, 0.5, 1)),
    material=gs.materials.Rigid(gravity_compensation=1.0)
)

# cam = scene.add_camera(
#     res    = (640, 480),
#     pos    = (3.5, 0.0, 2.5),
#     lookat = (0, 0, 0.5),
#     fov    = 30,
#     GUI    = False,
# )


[Genesis] [21:02:55] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <bf484a3>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [21:02:55] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <6aff0a0>, morph: <gs.morphs.MJCF(file='/home/vahid/envs/panda/lib/python3.10/site-packages/genesis/assets/xml/franka_emika_panda/panda.xml')>, material: <gs.materials.Rigid>.
[Genesis] [21:02:56] [WARNING] (MJCF) Tendon not supported
[Genesis] [21:02:56] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint1`
[Genesis] [21:02:56] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [21:02:56] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint2`
[Genesis] [21:02:56] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [21:02:56] [INFO] Appl

In [4]:
n_envs = 0
scene.build()
motors_dof = np.arange(7)
fingers_dof = np.arange(7, 9)
spinning_pads = np.arange(9, 11)

[Genesis] [21:02:57] [INFO] Building scene <baa42ad>...
[Genesis] [21:02:58] [WARNING] Reference robot position exceeds joint limits.
[Genesis] [21:02:58] [WARNING] Constraint solver time constant was increased to avoid numerical instability (from `0.005` to `0.02`). Decrease simulation timestep to avoid altering the original value.


[W 05/13/25 21:03:00.329 26143] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


[Genesis] [21:03:06] [INFO] Compiling simulation kernels...
[Genesis] [21:06:20] [INFO] Building visualizer...
[Genesis] [21:06:23] [INFO] Viewer created. Resolution: 1280×960, max_FPS: 60.


In [20]:
scene.reset()
qr = np.array([0.0000, -0.3002, 0.0000, -2.1991, 0.0000, 2.0071, 0.7854, 0.04, 0.04])

franka.ready_qpos = np.append(qr, [0, 0])
franka.set_qpos(franka.ready_qpos)
franka.control_dofs_position(franka.ready_qpos)
franka_MJCF.set_qpos(qr)
franka_MJCF.control_dofs_position(qr)

end_effector = franka.get_link('hand')
target_entity.set_pos(end_effector.get_pos())
target_entity.set_quat(end_effector.get_quat())


scene.step()

[Genesis] [21:20:07] [INFO] Resetting Scene <baa42ad>.
[Genesis] [21:20:07] [INFO] Running at 0.03 FPS.


In [6]:
cube_dq = adapter.get_dq(cube)
bbox = adapter.get_bbox(cube)
qs, s_axes = planner.screw_from_bbox(bbox=bbox)

In [7]:
q = qs[1] 
s_axis = s_axes[1]
q, s_axis

(array([ 0.75    ,  0.      , -0.000981], dtype=float32),
 array([0., 1., 0.], dtype=float32))

In [8]:
cube.get_qpos()

tensor([0.7000, 0.0000, 0.0140, 1.0000, 0.0000, 0.0000, 0.0000],
       device='cuda:0')

In [9]:
traj_pq = planner.generate_screw_trajectory(initial_pose=cube.get_qpos(),
                                  q=qs[1],
                                  s_axis=s_axes[1],
                                  output_type='p')

In [10]:
# target_quat = np.array([0, 1, 0, 0])
# center = np.array([0.5, 0, 0.5])
# r = 0.1
# damping = 1e-6
# diag = damping * np.eye(6)
# target_pos = center + np.array([1, 0, 0]) * r
# init_q = franka.inverse_kinematics(
#     link = end_effector,
#     pos  = target_pos,
#     quat = target_quat,
# )
# franka.set_qpos(init_q)
# franka.control_dofs_position(init_q)
# for i in range(100):
#     target_entity.set_qpos(np.concatenate([target_pos, target_quat]))
#     scene.step()

# for i in range(0, 500):
#     target_pos = center + np.array([np.cos(i / 360 * np.pi), np.sin(i / 360 * np.pi), 0]) * r

#     target_entity.set_qpos(np.concatenate([target_pos, target_quat]))

#     # Position error.
#     error_pos = target_pos - end_effector.get_pos().cpu().numpy()

#     # Orientation error.
#     ee_quat = end_effector.get_quat().cpu().numpy()
#     error_quat = gs.transform_quat_by_quat(gs.inv_quat(ee_quat), target_quat)
#     error_rotvec = gs.quat_to_rotvec(error_quat)

#     error = np.concatenate([error_pos, error_rotvec])

#     # jacobian
#     jac = franka.get_jacobian(link=end_effector).cpu().numpy()
#     dq = jac.T @ np.linalg.solve(jac @ jac.T + diag, error)
#     q = franka.get_qpos().cpu().numpy() + dq

#     # control
#     franka.control_dofs_position(q)
#     scene.step()


In [21]:

# get the end-effector link
end_effector = franka.get_link('hand')
end_effector_MJCF = franka_MJCF.get_link('hand')
above_box = cube.get_pos().cpu().numpy() + np.array([-cube_size[0]/2 + 0.02, 0, 0.11 + 0.1])
# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link = end_effector,
    pos  = above_box,
    quat = np.array([0, 1, 0, 0]),
    rot_mask = [False, True, False],
    dofs_idx_local = motors_dof
)
qpos_MJCF = franka_MJCF.inverse_kinematics(
    link = end_effector_MJCF,
    pos  = above_box + np.array([0.0, 2.0, 0.0]),
    quat = np.array([0, 1, 0, 0]),
)
# gripper open pos
qpos[7:8] = 0.04
qpos[-2:] = -1
qpos_MJCF[-2:] = 0.04
print(qpos)


tensor([-2.1854e-06,  6.1511e-01,  2.3524e-06, -1.7503e+00, -1.8258e-06,
         2.4070e+00,  7.8540e-01,  4.0000e-02,  4.0000e-02, -1.0000e+00,
        -1.0000e+00], device='cuda:0')


In [22]:
path = franka.plan_path(
    qpos_goal     = qpos,
    num_waypoints = 200, # 2s duration
)
path_MJCF = franka_MJCF.plan_path(
    qpos_goal     = qpos_MJCF,
    num_waypoints = 200, # 2s duration
)

[Genesis] [21:20:22] [WARNING] `qpos_start` exceeds joint limit. Relaxing joint limit to contain `qpos_start` for planning.
[Genesis] [21:20:23] [INFO] Path solution found successfully.
[Genesis] [21:20:23] [WARNING] `qpos_start` exceeds joint limit. Relaxing joint limit to contain `qpos_start` for planning.
[Genesis] [21:20:23] [INFO] Path solution found successfully.


In [23]:
# execute the planned path
for waypoint, waypoint_MJCF in zip(path, path_MJCF):
    franka.control_dofs_position(waypoint)
    franka_MJCF.control_dofs_position(waypoint_MJCF)
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()


[Genesis] [21:20:23] [INFO] Running at 0.03 FPS.
[Genesis] [21:20:23] [INFO] Running at 0.03 FPS.
[Genesis] [21:20:23] [INFO] Running at 0.03 FPS.
[Genesis] [21:20:23] [INFO] Running at 0.03 FPS.
[Genesis] [21:20:23] [INFO] Running at 0.03 FPS.
[Genesis] [21:20:24] [INFO] Running at 0.04 FPS.
[Genesis] [21:20:24] [INFO] Running at 0.04 FPS.
[Genesis] [21:20:24] [INFO] Running at 0.04 FPS.
[Genesis] [21:20:24] [INFO] Running at 0.04 FPS.
[Genesis] [21:20:24] [INFO] Running at 0.05 FPS.
[Genesis] [21:20:24] [INFO] Running at 0.05 FPS.
[Genesis] [21:20:24] [INFO] Running at 0.05 FPS.
[Genesis] [21:20:24] [INFO] Running at 0.05 FPS.
[Genesis] [21:20:24] [INFO] Running at 0.06 FPS.
[Genesis] [21:20:24] [INFO] Running at 0.06 FPS.
[Genesis] [21:20:24] [INFO] Running at 0.06 FPS.
[Genesis] [21:20:24] [INFO] Running at 0.06 FPS.
[Genesis] [21:20:24] [INFO] Running at 0.07 FPS.
[Genesis] [21:20:24] [INFO] Running at 0.07 FPS.
[Genesis] [21:20:24] [INFO] Running at 0.08 FPS.
[Genesis] [21:20:24]

In [14]:
franka.set_qpos(qpos)
franka.control_dofs_position(qpos[:-4], motors_dof)
franka.control_dofs_force(np.array([-0, -0]), fingers_dof)
# allow robot to reach the last waypoint
for i in range(100):
    target_entity.set_pos(end_effector.get_pos().cpu().numpy() + np.array([0, 0, -0.1034]))
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

[Genesis] [21:06:57] [INFO] Running at 28.08 FPS.
[Genesis] [21:06:57] [INFO] Running at 28.26 FPS.
[Genesis] [21:06:57] [INFO] Running at 28.38 FPS.
[Genesis] [21:06:57] [INFO] Running at 28.65 FPS.
[Genesis] [21:06:57] [INFO] Running at 29.12 FPS.
[Genesis] [21:06:57] [INFO] Running at 29.30 FPS.
[Genesis] [21:06:57] [INFO] Running at 29.47 FPS.
[Genesis] [21:06:57] [INFO] Running at 29.52 FPS.
[Genesis] [21:06:57] [INFO] Running at 29.59 FPS.
[Genesis] [21:06:57] [INFO] Running at 29.78 FPS.
[Genesis] [21:06:57] [INFO] Running at 30.01 FPS.
[Genesis] [21:06:57] [INFO] Running at 30.02 FPS.
[Genesis] [21:06:57] [INFO] Running at 29.90 FPS.
[Genesis] [21:06:57] [INFO] Running at 29.78 FPS.
[Genesis] [21:06:57] [INFO] Running at 29.74 FPS.
[Genesis] [21:06:57] [INFO] Running at 29.81 FPS.
[Genesis] [21:06:57] [INFO] Running at 29.88 FPS.
[Genesis] [21:06:57] [INFO] Running at 30.12 FPS.
[Genesis] [21:06:58] [INFO] Running at 30.03 FPS.
[Genesis] [21:06:58] [INFO] Running at 30.05 FPS.


In [24]:
side_box = above_box + np.array([0.0, 0.0, -0.1])


# reach
qpos_reach = franka.inverse_kinematics(
    link = end_effector,
    pos  = side_box,
    quat = np.array([0, 1, 0, 0]),
    rot_mask = [False, True, False]
)
qpos_reach_MJCF = franka_MJCF.inverse_kinematics(
    link = end_effector_MJCF,
    pos  = side_box + np.array([0.0, 2.0, 0.0]),
    quat = np.array([0, 1, 0, 0]),
)


In [25]:
path = franka.plan_path(
    qpos_goal     = qpos_reach,
    num_waypoints = 100, # 1s duration
)
path_MJCF = franka_MJCF.plan_path(
    qpos_goal     = qpos_reach_MJCF,
    num_waypoints = 100, # 1s duration
)

for waypoint, waypoint_MJCF in zip(path, path_MJCF):
    franka.control_dofs_position(waypoint)
    franka_MJCF.control_dofs_position(waypoint_MJCF)
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

[Genesis] [21:20:38] [WARNING] `qpos_start` exceeds joint limit. Relaxing joint limit to contain `qpos_start` for planning.


[Genesis] [21:20:38] [INFO] Path solution found successfully.
[Genesis] [21:20:39] [INFO] Path solution found successfully.
[Genesis] [21:20:39] [INFO] Running at 2.93 FPS.
[Genesis] [21:20:39] [INFO] Running at 3.07 FPS.
[Genesis] [21:20:39] [INFO] Running at 3.22 FPS.
[Genesis] [21:20:39] [INFO] Running at 3.38 FPS.
[Genesis] [21:20:39] [INFO] Running at 3.54 FPS.
[Genesis] [21:20:39] [INFO] Running at 3.70 FPS.
[Genesis] [21:20:39] [INFO] Running at 3.87 FPS.
[Genesis] [21:20:39] [INFO] Running at 4.05 FPS.
[Genesis] [21:20:39] [INFO] Running at 4.24 FPS.
[Genesis] [21:20:39] [INFO] Running at 4.43 FPS.
[Genesis] [21:20:39] [INFO] Running at 4.63 FPS.
[Genesis] [21:20:39] [INFO] Running at 4.84 FPS.
[Genesis] [21:20:39] [INFO] Running at 5.06 FPS.
[Genesis] [21:20:39] [INFO] Running at 5.29 FPS.
[Genesis] [21:20:39] [INFO] Running at 5.52 FPS.
[Genesis] [21:20:39] [INFO] Running at 5.77 FPS.
[Genesis] [21:20:39] [INFO] Running at 6.02 FPS.
[Genesis] [21:20:39] [INFO] Running at 6.28

In [26]:
franka.control_dofs_force(np.array([-5, -5]), fingers_dof)
franka_MJCF.set_qpos(qpos_reach_MJCF)
franka_MJCF.control_dofs_position(qpos_reach_MJCF[:-2], motors_dof)
franka_MJCF.control_dofs_force(np.array([-5, -5]), fingers_dof)

for i in range(100):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()


[Genesis] [21:21:26] [INFO] Running at 0.48 FPS.
[Genesis] [21:21:26] [INFO] Running at 0.50 FPS.
[Genesis] [21:21:26] [INFO] Running at 0.53 FPS.
[Genesis] [21:21:26] [INFO] Running at 0.55 FPS.
[Genesis] [21:21:26] [INFO] Running at 0.58 FPS.
[Genesis] [21:21:26] [INFO] Running at 0.61 FPS.
[Genesis] [21:21:27] [INFO] Running at 0.64 FPS.
[Genesis] [21:21:27] [INFO] Running at 0.67 FPS.
[Genesis] [21:21:27] [INFO] Running at 0.70 FPS.
[Genesis] [21:21:27] [INFO] Running at 0.74 FPS.
[Genesis] [21:21:27] [INFO] Running at 0.77 FPS.
[Genesis] [21:21:27] [INFO] Running at 0.81 FPS.
[Genesis] [21:21:27] [INFO] Running at 0.85 FPS.
[Genesis] [21:21:27] [INFO] Running at 0.89 FPS.
[Genesis] [21:21:27] [INFO] Running at 0.93 FPS.
[Genesis] [21:21:28] [INFO] Running at 0.97 FPS.
[Genesis] [21:21:28] [INFO] Running at 1.01 FPS.
[Genesis] [21:21:28] [INFO] Running at 1.06 FPS.
[Genesis] [21:21:28] [INFO] Running at 1.11 FPS.
[Genesis] [21:21:28] [INFO] Running at 1.16 FPS.
[Genesis] [21:21:28]

In [27]:

# lift
# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link = end_effector,
    pos  = above_box,
    quat = np.array([0, 1, 0, 0]),
)
qpos_MJCF = franka_MJCF.inverse_kinematics(
    link = end_effector_MJCF,
    pos  = above_box + np.array([0.0, 2.0, 0.0]),
    quat = np.array([0, 1, 0, 0]),
)

franka.control_dofs_position(qpos[:-4], motors_dof)
franka_MJCF.control_dofs_position(qpos_MJCF[:-2], motors_dof)

for i in range(200):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

[Genesis] [21:21:46] [INFO] Running at 1.65 FPS.
[Genesis] [21:21:46] [INFO] Running at 1.72 FPS.
[Genesis] [21:21:46] [INFO] Running at 1.79 FPS.
[Genesis] [21:21:46] [INFO] Running at 1.88 FPS.
[Genesis] [21:21:47] [INFO] Running at 1.96 FPS.
[Genesis] [21:21:47] [INFO] Running at 2.04 FPS.
[Genesis] [21:21:47] [INFO] Running at 2.12 FPS.
[Genesis] [21:21:47] [INFO] Running at 2.21 FPS.
[Genesis] [21:21:47] [INFO] Running at 2.30 FPS.
[Genesis] [21:21:47] [INFO] Running at 2.40 FPS.
[Genesis] [21:21:47] [INFO] Running at 2.49 FPS.
[Genesis] [21:21:47] [INFO] Running at 2.59 FPS.
[Genesis] [21:21:47] [INFO] Running at 2.69 FPS.
[Genesis] [21:21:47] [INFO] Running at 2.80 FPS.
[Genesis] [21:21:48] [INFO] Running at 2.90 FPS.
[Genesis] [21:21:48] [INFO] Running at 3.01 FPS.
[Genesis] [21:21:48] [INFO] Running at 3.12 FPS.
[Genesis] [21:21:48] [INFO] Running at 3.23 FPS.
[Genesis] [21:21:48] [INFO] Running at 3.35 FPS.
[Genesis] [21:21:48] [INFO] Running at 3.47 FPS.
[Genesis] [21:21:48]